# Домашнее задание: класс линейной регрессии

## Импорт библиотек, установка констант

In [2]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

## Интерфейс Scikit-Learn

Scikit-Learn (`sklearn`)- библиотека, в которой реализованы практически все используемые сегодня алгоритмы машинного обучения.

Для реализации алгоритмов машинного обучения в `sklearn` всегда используется один интерфейс - класс с функциями `fit(X, y)` для обучения модели по обучающей выборке `X`, `y` и `predict(X)` для возвращения предсказаний на выборке `X`. При создании класса можно указывать дополнительные параметры, влияющие на работу алгоритма машинного обучения.

## Линейная регрессия

Реализуйте класс линейной регрессии c L2-регуляризацией со следующей логикой:
*   При создании класса задайте коэффициент регуляризации `reg_coef`, равный по умолчанию нулю, а также веса (=None)
*   Задача функции `fit` - по выборке `X` и `y` найти веса `w` и сохранить их внутри класса в `self.w`:  
$w = (X^TX + \lambda I)^{-1}X^Ty,$
где $\lambda$ - коэффициент регуляризации, $I$ - единичная матрица.
  
P.S. Формула верна только при наличии вектора признаков, равного 1 - поэтому для вашего удобства мы уже добавили его в класс.

*   Задача функции `predict` - по весам `self.w` и `X` вернуть предсказания  


In [5]:
from numpy import linalg

class LinearRegressor:
    def __init__(self, reg_coef = 0.0) -> None:
        self.lambda_ = reg_coef
        self.weights = None

    def fit(self, X_train: np.array, y_train: np.array) -> None:
        X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        self.weights = linalg.inv(X_train.T@X_train + self.lambda_*np.ones((X_train.T.shape[0], X_train.shape[1])))@X_train.T@y_train.ravel()

    def predict(self, X_test: np.array) -> np.array:
        X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

        pred = X_test@self.weights

        return pred

Если бы не использовали класс, нам пришлось бы передавать веса `w` в функцию `predict()` каждый раз, когда мы захотели бы сделать предсказания, а так они хранятся внутри класса. Это особенно удобно, если таких вспомогательных переменных много.

Будем тестировать ваш класс на датасете о стоимости домов в Калифорнии.

In [6]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

Разобъем данные на тренировочную и тестовую часть.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
    )

Масштабируем данные.

In [8]:
sc = StandardScaler()

sc.fit(X_train)

X_train = pd.DataFrame(sc.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

## Задание 1

Обучите вашу линейную регрессию (без регуляризации) на тренировочных данных. Выведите на экран максимальный по модулю вес (со знаком). Ответ округлите до сотых.

In [10]:
model = LinearRegressor()
model.fit(X_train, y_train)
model.weights

array([ 2.07034892e+00,  8.52108147e-01,  1.20655325e-01, -3.02105554e-01,
        3.48605752e-01, -1.64465406e-03, -4.11635631e-02, -8.93146971e-01,
       -8.67840463e-01])

In [11]:
max(abs(model.weights))

2.0703489205426377

## Задание 2

Сделайте прогноз на тестовых данных и выведите на экран значение метрики $R^2$ на тесте.  
Ответ округлите до тысячных.

In [14]:
# ваш код здесь
from sklearn.metrics import r2_score

Y_pred = model.predict(X_test)

In [16]:
Y_pred.shape

(5160,)

In [17]:
y_test.shape

(5160,)

In [19]:
r2_score(y_test, Y_pred)

0.5910509795491352

## Задание 3

Теперь обучите линейную регрессию с коэффициентом регуляризации $\alpha = 100$ на тренировочных данных.

Чему теперь равен наибольший по модулю вес? Ответ округлите до сотых.

In [21]:
# ваш код здесь
model_2 = LinearRegressor(reg_coef=100)
model_2.fit(X_train, y_train)

In [23]:
round(max(abs(model_2.weights)),3)

2.064

## Задание 4

Для модели с регуляризацией сделайте прогноз на тестовых данных и выведите на экран значение метрики $R^2$ на тесте.  
Ответ округлите до тысячных.

In [24]:
# ваш код здесь
Y_pred2 = model_2.predict(X_test)
r2_score(y_test, Y_pred2)

0.5923211097894452